In [ ]:
%matplotlib qt
import xarray as xr
import matplotlib.pyplot as plt

plt.rcParams.update({
    "font.family": "serif",
    "font.size": 10,              # base (LaTeX will scale the figure anyway)
    "axes.labelsize": 10,
    "axes.titlesize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
})
import cartopy.crs as ccrs
import numpy as np
import cdsapi

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.path import Path
import matplotlib.patches as mpatches
from cartopy.util import add_cyclic_point

In [ ]:
import cdsapi


def download_data(day, month, year, time, pressure_level):
    c = cdsapi.Client()
    c.retrieve(
        "reanalysis-era5-pressure-levels",
        {
            "product_type": "reanalysis",
            "format": "netcdf",
            "variable": ["geopotential", "u_component_of_wind", "v_component_of_wind"],
            "pressure_level": pressure_level,
            "year": str(year),
            "month": str(month).zfill(2),
            "day": str(day).zfill(2),
            "time": time,
        },
        f"data/data_{year}_{month}_{day}_{time}_{pressure_level}.nc",
    )

In [ ]:
def plot(path):

    ds = xr.open_dataset(f"data/{path}.nc")

    ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))
    ds = ds.sortby("longitude")  # Important!

    u = ds["u"]
    v = ds["v"]
    z = ds["z"] / 9.80665

    stride = 10

    # Slice data in both lat and lon
    z_sub = z.squeeze()[::stride, ::stride]
    u_sub = u.squeeze()[::stride, ::stride]
    v_sub = v.squeeze()[::stride, ::stride]
    wind_speed_sub = np.sqrt(u_sub**2 + v_sub**2)

    lon_sub = z.longitude[::stride]
    lat_sub = z.latitude[::stride]
    wind_speed = np.sqrt(u**2 + v**2)

    # Define the color stops
    colors = ["white", "blue", "green", "yellow", "red"]
    custom_cmap = mcolors.LinearSegmentedColormap.from_list(
        "custom_wbgr", colors, N=256
    )

    # proj = ccrs.LambertConformal(central_longitude=-30, central_latitude=50)
    proj = ccrs.LambertConformal(central_longitude=60, central_latitude=40)
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": proj})
    # Set extent: [lon_min, lon_max, lat_min, lat_max]
    # ax.set_extent(
    #     [-90, 30, 20, 75], crs=ccrs.PlateCarree()
    # )  # Covers NAmerica, Atlantic, Europe
    ax.set_extent([0, 120, 10, 70], crs=ccrs.PlateCarree())
    # ax.set_extent([0, 120, 10, 70], crs=ccrs.PlateCarree())
    # Plot data (wind, geopotential etc. — assume u, v, z, wind_speed already extracted and sliced)
    cf = ax.contourf(
        lon_sub,
        lat_sub,
        wind_speed_sub,
        levels=20,
        cmap=custom_cmap,
        extend="max",
        alpha=0.2,
        transform=ccrs.PlateCarree(),
    )

    cs = ax.contour(
        lon_sub,
        lat_sub,
        z_sub,
        levels=20,
        colors="black",
        linewidths=0.5,
        transform=ccrs.PlateCarree(),
    )
    ax.clabel(cs, inline=True, fontsize=8, fmt="%d")

    lon_mesh, lat_mesh = np.meshgrid(lon_sub.values, lat_sub.values)
    stride2 = 1
    ax.quiver(
        lon_mesh[::stride2, ::stride2],
        lat_mesh[::stride2, ::stride2],
        u_sub.values[::stride2, ::stride2],
        v_sub.values[::stride2, ::stride2],
        scale=900,
        alpha=0.7,
        transform=ccrs.PlateCarree(),
    )

    # Add map features
    ax.coastlines(resolution="50m", color="black", linewidth=1)
    # ax.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='black', linestyle='-', linewidth=0.8)
    ax.add_feature(cfeature.LAND, facecolor="#C69930", alpha=0.25)  # or 'lightgray'
    ax.add_feature(cfeature.OCEAN, facecolor="lightblue", alpha=0.25)  # or 'lightblue'

    # Add longitude/latitude gridlines
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color="gray", alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    # gl.xlabel_style = {"size": 12}
    # gl.ylabel_style = {"size": 12}

    plt.title(
        f"{int(u.pressure_level.values)} hPa: Wind Speed (m/s), Geopotential Height (gpm)",
        fontsize=10,
    )
    plt.colorbar(
        cf,
        ax=ax,
        orientation="horizontal",
        pad=0.05,
        label="Wind Speed (m/s)",
        shrink=0.5,
    )
    plt.tight_layout()
    plt.savefig(f"../images/{path}.jpg", dpi=150)
    # plt.savefig(f"../images/{path}.pdf")
    # plt.show()
    plt.close()


year = 2010
month = 8
day = [1,2,3,4,5,6,7,8]  # Days of the month
t = ["12:00"]
pl = "500"  # Pressure level in hPa
for d in day:
    for ti in t:
        path = f"data_{year}_{month:1d}_{d:1d}_{ti}_{pl}"
        plot(path)

In [ ]:
import os
import glob

# Replace all ':' with '_' in file names under the 'data' directory
for file_path in glob.glob("../images/*"):
    new_file_path = file_path.replace(":", "_")
    if new_file_path != file_path:
        os.rename(file_path, new_file_path)

In [ ]:
def plot_polar(path):
    ds = xr.open_dataset(f"data/{path}.nc")
    ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))
    ds = ds.sortby("longitude")

    u = ds["u"]
    v = ds["v"]
    z = ds["z"] / 9.80665

    # Clip to Northern Hemisphere only
    ds = ds.sel(latitude=slice(90, 30))
    u = u.sel(latitude=slice(90, 30))
    v = v.sel(latitude=slice(90, 30))
    z = z.sel(latitude=slice(90, 30))

    stride = 10
    z_sub = z.squeeze()[::stride, ::stride]
    u_sub = u.squeeze()[::stride, ::stride]
    v_sub = v.squeeze()[::stride, ::stride]
    lon_sub = z.longitude[::stride]
    lat_sub = z.latitude[::stride]
    wind_speed_sub = np.sqrt(u_sub**2 + v_sub**2)

    wind_speed_sub, lon_cyclic = add_cyclic_point(wind_speed_sub, coord=lon_sub)
    u_sub, _ = add_cyclic_point(u_sub, coord=lon_sub)
    v_sub, _ = add_cyclic_point(v_sub, coord=lon_sub)
    z_sub, _ = add_cyclic_point(z_sub, coord=lon_sub)

    colors = ["white", "blue", "green", "yellow", "red"]
    custom_cmap = mcolors.LinearSegmentedColormap.from_list(
        "custom_wbgr", colors, N=256
    )

    proj = ccrs.NorthPolarStereo()
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"projection": proj})
    ax.set_extent([-180, 180, 30, 90], crs=ccrs.PlateCarree())

    theta = np.linspace(0, 2 * np.pi, 100)
    circle = np.vstack([0.5 + 0.5 * np.cos(theta), 0.5 + 0.5 * np.sin(theta)]).T
    boundary = mpatches.PathPatch(
        Path(circle), transform=ax.transAxes, facecolor="none"
    )

    ax.set_boundary(boundary.get_path(), transform=boundary.get_transform())
    cf = ax.contourf(
        lon_cyclic,
        lat_sub,
        wind_speed_sub,
        levels=20,
        cmap=custom_cmap,
        extend="max",
        alpha=0.2,
        transform=ccrs.PlateCarree(),
    )

    cs = ax.contour(
        lon_cyclic,
        lat_sub,
        z_sub,
        levels=10,
        colors="black",
        linewidths=0.8,
        transform=ccrs.PlateCarree(),
    )
    ax.clabel(cs, inline=True, fontsize=8, fmt="%d")

    lon_mesh, lat_mesh = np.meshgrid(lon_cyclic, lat_sub)
    stride2 = 1
    # ax.quiver(
    #     lon_mesh[::stride2, ::stride2],
    #     lat_mesh[::stride2, ::stride2],
    #     u_sub.values[::stride2, ::stride2],
    #     v_sub.values[::stride2, ::stride2],
    #     scale=900,
    #     alpha=0.7,
    #     transform=ccrs.PlateCarree(),
    # )
    # ax.streamplot(
    #     lon_mesh,
    #     lat_mesh,
    #     u_sub,
    #     v_sub,
    #     transform=ccrs.PlateCarree(),
    #     color="black",
    #     linewidth=0.4,
    #     density=2,
    # )

    ax.coastlines(resolution="50m", color="black", linewidth=1)
    ax.add_feature(cfeature.LAND, facecolor="#C69930", alpha=0.25)
    ax.add_feature(cfeature.OCEAN, facecolor="lightblue", alpha=0.25)
    pressure_level = int(u.pressure_level.values.item())

    plt.title(f"{pressure_level} hPa: Wind & Geopotential (Northern Hemisphere)")
    plt.colorbar(
        cf,
        ax=ax,
        orientation="horizontal",
        pad=0.05,
        label="Wind Speed (m/s)",
        shrink=0.5,
    )
    plt.tight_layout()
    plt.savefig(f"../images/weather/{path}_polar.png")
    plt.savefig(f"../images/weather/{path}_polar.pdf")
    plt.close()


year = 2025
month = 5
day = 10
t = "12:00"
pl = "500"  # Pressure level in hPa
path = f"data_{year}_{month:1d}_{day:1d}_{t}_{pl}"
plot_polar(path)

In [ ]:
# Define parameters
year = 2010
month = 8
days = [1,2]
times = ["12:00"]
pressure_levels = ["500"]  # ["850", "500", "200"]

# Download ERA5 pressure level data
for day in days:
    for time in times:
        for level in pressure_levels:
            download_data(
                day=day, month=month, year=year, time=time, pressure_level=level
            )

In [ ]:
year = 2010
month = 8
days = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # July 20–31 + August 1–10
times = ["12:00"]
pressure_levels = ["500"]

for day in days:
    for time in times:
        for level in pressure_levels:
            actual_month = month if day <= 31 else 8
            actual_day = day if day <= 31 else day - 31
            download_data(
                day=actual_day,
                month=actual_month,
                year=year,
                time=time,
                pressure_level=level,
            )

In [ ]:
year = 2010
month = 7
days = range(20, 32)
times = ["12:00"]
pressure_levels = ["500"]

for day in days:
    for pl in pressure_levels:
        for t in times:
            path = f"data_{year}_{month:1d}_{day:1d}_{t}_{pl}"
            plot_polar(path)